In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pathlib
import pathlib
import os
import shutil
import gdal
import subprocess

import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_DATA = PATH_ROOT + '/data/01_raw/'

# links
LINK_NE_RIVERS = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines_scale_rank.zip"
LINK_NE_RIVERS_EU = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_europe.zip"

LINK_SRTM_DIR = 'https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Eurasia/'

DIR_OUT_SRTM  = PATH_DATA + '/srtm/'

In [4]:
'''
Download google spreadsheet tables
'''
LINK_ACTIVITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=0&single=true&output=csv'
LINK_LOCALITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=121203183&single=true&output=csv'

urllib.request.urlretrieve(LINK_ACTIVITIES, PATH_DATA + 'activities.csv')
urllib.request.urlretrieve(LINK_LOCALITIES, PATH_DATA + 'localities.csv')


('/home/adam/projects/itinerary-analysis_premysl-otakar/src/processing/../../data/01_raw/localities.csv',
 <http.client.HTTPMessage at 0x7f2eec5eb4c0>)

In [4]:
'''
Download and merge srtm 
'''

html_page = urllib.request.urlopen(LINK_SRTM_DIR)
soup = BeautifulSoup(html_page)

shutil.rmtree(DIR_OUT_SRTM)
os.makedirs(DIR_OUT_SRTM)

srtm_to_download = []
for li, link in enumerate(soup.findAll("a")):
    
    SRTM_NAME = link.get("href")
    try:
        y = int(SRTM_NAME[1:3])
        x = int(SRTM_NAME[4:7])

        x_WE = SRTM_NAME[3:4]
        y_SN = SRTM_NAME[0:1]

        if y_SN == 'N' and x_WE == 'E' and x > 5 and x < 23 and y > 42 and y < 55:
            srtm_to_download.append(SRTM_NAME)
    except:
        print('problem parsing', SRTM_NAME)
            
            
def download_srtm(link_srtm):
    try:
        #print('downlaoding', link_srtm)
        urllib.request.urlretrieve(LINK_SRTM_DIR + link_srtm, DIR_OUT_SRTM + link_srtm)
        shutil.unpack_archive(DIR_OUT_SRTM + link_srtm, DIR_OUT_SRTM)
    except: 
        print('failed downloading', link_srtm)

# for srtm in srtm_to_download:
#     download_srtm(srtm)

# download all filtered srtm
Parallel(n_jobs=num_cores)(
    delayed(download_srtm)(srtm) for srtm in srtm_to_download
)

# remove zip files
for file in os.listdir(DIR_OUT_SRTM):
    if file.endswith(".zip"):
        os.remove(DIR_OUT_SRTM + file)


HTTPError: HTTP Error 404: Not Found

In [18]:
'''
Merge downloaded srtm files
'''
# all remaining srtm files
source_srtm = [DIR_OUT_SRTM + file for file in os.listdir(DIR_OUT_SRTM)]

# merge srtm files
merge_command = ["gdal_merge.py", "-o", DIR_OUT_SRTM + "merged.tif"] + source_srtm
subprocess.check_output(merge_command)

b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'